In [15]:
using JuMP, Gurobi, CSV, DataFrames, JSON, LightGraphs
global α = 0.95
global accept_rate = 0.8
data = CSV.read("test.csv",DataFrame)
println(data)
global L = data[1,"customers"]
global Big_M = 100

1×6 DataFrame
 Row │ customers  vehicles  capacity  demand                             edges                              x_bar                             
     │ Int64      Int64     Int64     String                             String                             String                            
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        10         2        10  [0, 1, 3, 2, 4, 1, 4, 2, 3, 2, 3]  [[1, 2], [2, 1], [1, 9], [9, 1],…  [1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1…


100

In [16]:
function read_Kth_data(data, k)

    customers = data[k,"customers"]

    K = data[k,"vehicles"]

    C = data[k,"capacity"]

    edges_raw = JSON.parse(data[k,"edges"])
    edges_raw  = Vector{Vector{Int64}}(edges_raw)

    sol_raw = JSON.parse(data[k,"x_bar"])
    sol_raw = Vector{Float64}(sol_raw)

    demands = JSON.parse(data[k,"demand"])
    demands = Vector{Int64}(demands)

    edges = Vector{Vector{Int64}}()
    #sol = Vector{Tuple{Int64,Int64,Float64}}()
    sol = Dict()

    for i in 1:2:length(edges_raw)  # get support graph s.t Xij >0
        if sol_raw[i] > 0
            push!(edges, edges_raw[i])
            sol["($(edges_raw[i][1]),$(edges_raw[i][2]))"] =  sol_raw[i]
            #push!(sol, (edges_raw[i][1],edges_raw[i][2],sol_raw[i]))
        end
    end

    input = [customers, K, C, demands, edges, sol]
    
    return input
end

read_Kth_data (generic function with 1 method)

In [17]:
function support_graph(input)
    #g = SimpleGraph(input[1]+1)

    #for i in 1:length(input[5])
    #    add_edge!(g,input[5][i][1],input[5][i][2])
    #end

    g = Dict("$i" => [] for i in 1:input[1]+1)
    

    #println(g)

    for i in 1:length(input[5])
        push!(g["$(input[5][i][1])"],input[5][i][2])
        push!(g["$(input[5][i][2])"],input[5][i][1])
    end

    return g
end


support_graph (generic function with 1 method)

In [18]:
function initial_sol_min_subset(input, M, C)
    sol_initial_y = Vector{Vector{Int64}}(undef,input[1]+1) #yi's value& di's value
    #sol_initial_w = input[5] #i,j,Wij
    sol_initial_w = Dict()
    demand = input[4]
    sorted_nodes = sortperm(demand, rev=true)
    sum_demand = 0

    for num in 1:length(sorted_nodes)
        if sorted_nodes[num] == 1
            sol_initial_y[sorted_nodes[num]] = [0,0]
            continue
        end
       
        if sum_demand < M*C + 1
            sol_initial_y[sorted_nodes[num]] = [1,demand[sorted_nodes[num]]]
            sum_demand = sum_demand + demand[sorted_nodes[num]]
        else
            sol_initial_y[sorted_nodes[num]] = [0,demand[sorted_nodes[num]]]
        end
    end

    for i in 1:length(input[5])
        key = "($(input[5][i][1]),$(input[5][i][2]))"
        if sol_initial_y[input[5][i][1]][1] == 1 && sol_initial_y[input[5][i][2]][1] == 0 
            value = 1
        elseif sol_initial_y[input[5][i][1]][1] == 0 && sol_initial_y[input[5][i][2]][1] == 1
            value = 1
        else
            value = 0 
        end
        sol_initial_w[key] = value
    end
    



    #for i in 1:length(sol_initial_w)
    #    if sol_initial_y[sol_initial_w[i][1]][1] == 1 && sol_initial_y[sol_initial_w[i][2]][1] == 0
    #        push!(sol_initial_w[i],1)
    #    elseif sol_initial_y[sol_initial_w[i][1]][1] == 0 && sol_initial_y[sol_initial_w[i][2]][1] == 1
    #        push!(sol_initial_w[i],1)
    #    else
    #        push!(sol_initial_w[i],0)
    #    end
    #end

    ##println("X0: ", [sol_initial_y,sol_initial_w,sum_demand])

    return [sol_initial_y,sol_initial_w,sum_demand]
 
end

initial_sol_min_subset (generic function with 1 method)

In [19]:
function initial_sol_max_subset(input, M, C)
    sol_initial_y = Vector{Vector{Int64}}(undef,input[1]+1) #yi's value& di's value
    sol_initial_w = Dict()
    demand = input[4]
    -
    
    for num in 1:length(sol_initial_y)
       if num == 1
            sol_initial_y[num] = [0,0]
       else
            sol_initial_y[num] = [1,demand[num]]
       end
    end
    
    for i in 1:length(input[5])
        key = "($(input[5][i][1]),$(input[5][i][2]))"
        if sol_initial_y[input[5][i][1]][1] == 1 && sol_initial_y[input[5][i][2]][1] == 0 
            value = 1
        elseif sol_initial_y[input[5][i][1]][1] == 0 && sol_initial_y[input[5][i][2]][1] == 1
            value = 1
        else
            value = 0 
        end
        sol_initial_w[key] = value
    end

    return [sol_initial_y,sol_initial_w,sum(demand)]
 
end


initial_sol_max_subset (generic function with 1 method)

In [20]:
function f(x_cur, coefficient)
    Wij = deepcopy(x_cur[2][1]) #O(n)
    #Xij = deepcopy(coefficient) #O(n)
    Obj_value = 0

    #println(length(Xij))
    for (key,value) in coefficient #O(n)
        Obj_value = Obj_value + value*Wij["$key"]
    end

    Obj_value = Obj_value

    ##println("Obj_value: ",Obj_value)
    return Obj_value 
end

f (generic function with 1 method)

In [21]:
function stop(T)
    stop = 0
    if(T < 0.01)
        stop = 1
    end
    #println("T: ",T)
    return stop
end

stop (generic function with 1 method)

In [22]:
function get_Wij(x_cur)
    sol_y = deepcopy(x_cur[1]) #O(n)
    sol_w = deepcopy(x_cur[2])

    for i in 1:length(sol_w) #O(n)
        if sol_y[sol_w[i][1]][1] == 1 && sol_y[sol_w[i][2]][1] == 0
            sol_w[i][3] = 1
        elseif sol_y[sol_w[i][1]][1] == 0 && sol_y[sol_w[i][2]][1] == 1
            sol_w[i][3] = 1
        else
            sol_w[i][3] = 0
        end
    end
    
    return sol_w
end

get_Wij (generic function with 1 method)

In [23]:
function neighbor_sol(x_cur, M, C, g, obj_cur, coeffieient, T_cur, mod) #1-fliping
    #y_cur = deepcopy(x_cur[1]) #O(n)
    #w_cur = deepcopy(x_cur[2]) #O(n)

    y_cur = x_cur[1][2]
    w_cur = x_cur[2][2]
    demand = deepcopy(x_cur[3])
    obj_new = deepcopy(obj_cur)
    index_fliped = rand(2:length(y_cur))
    penalty = 0
    

    if y_cur[index_fliped][1] == 1
        y_cur[index_fliped][1] = 0
        demand = demand - y_cur[index_fliped][2]
    else
        y_cur[index_fliped][1] = 1
        demand = demand + y_cur[index_fliped][2]
    end

    if demand < M*C + 1
        penalty = Big_M*(M*C + 1 - demand)
    else
        penalty = 0
    end

    ##println("x_cur: $(x_cur[1])")
    ##println("y_cur: $y_cur")

    neighbor_list = g["$index_fliped"]

    #println(length(neighbor_list))
    #println("neighbor_list: ",neighbor_list)
    #println((neighbor_list[1],index_fliped))
    edges_changed = Vector{Tuple{Int64, Int64}}()
    for i in 1:length(neighbor_list)
        if haskey(w_cur,"($(index_fliped),$(neighbor_list[i]))") == true
            if w_cur["($(index_fliped),$(neighbor_list[i]))"] == 1
                w_cur["($(index_fliped),$(neighbor_list[i]))"] = 0
                obj_new = obj_new - coeffieient["($(index_fliped),$(neighbor_list[i]))"]
                ##println(coeffieient["($(index_fliped),$(neighbor_list[i]))"])
                push!(edges_changed,(index_fliped,neighbor_list[i]))
            else
                w_cur["($(index_fliped),$(neighbor_list[i]))"] = 1
                obj_new = obj_new + coeffieient["($(index_fliped),$(neighbor_list[i]))"]
                ##println(coeffieient["($(index_fliped),$(neighbor_list[i]))"])
                push!(edges_changed,(index_fliped,neighbor_list[i]))
                
            end
        else
            if w_cur["($(neighbor_list[i]),$(index_fliped))"] == 1
                w_cur["($(neighbor_list[i]),$(index_fliped))"] = 0
                obj_new = obj_new - coeffieient["($(neighbor_list[i]),$(index_fliped))"]
                ##println(coeffieient["($(neighbor_list[i]),$(index_fliped))"])
                push!(edges_changed,(neighbor_list[i],index_fliped))
                
            else
                w_cur["($(neighbor_list[i]),$(index_fliped))"] = 1
                obj_new = obj_new + coeffieient["($(neighbor_list[i]),$(index_fliped))"]
                ##println(coeffieient["($(neighbor_list[i]),$(index_fliped))"])
                push!(edges_changed,(neighbor_list[i],index_fliped))
            end
        end
    end
    ##println("obj_cur: $(obj_cur)")
    ##println("obj_new: $(obj_new+ penalty)")
    ΔObj_value = (obj_new + penalty) - obj_cur
    ##println("ΔObj_value: $(ΔObj_value)")
    if mod == 1
        if ΔObj_value <= 0 || rand() < exp(-ΔObj_value/T_cur)
            x_cur[1][1][index_fliped][1] = y_cur[index_fliped][1]        #x_cur[1][1] = y_cur
            for i in 1:length(edges_changed)        #x_cur[2][1] = w_cur
                x_cur[2][1]["($(edges_changed[i][1]),$(edges_changed[i][2]))"] = w_cur["($(edges_changed[i][1]),$(edges_changed[i][2]))"]
            end
            x_cur[3] = demand
            obj_cur = obj_new
            if ΔObj_value <= 0
                ##println("a")
            else
                ##println("b")
            end
        else
            ##println("c")
            y_cur[index_fliped][1] = x_cur[1][1][index_fliped][1]
            for i in 1:length(edges_changed)        
                w_cur["($(edges_changed[i][1]),$(edges_changed[i][2]))"] = x_cur[2][1]["($(edges_changed[i][1]),$(edges_changed[i][2]))"] 
            end
        end
    else
        y_cur[index_fliped][1] = x_cur[1][1][index_fliped][1]
        for i in 1:length(edges_changed)        
            w_cur["($(edges_changed[i][1]),$(edges_changed[i][2]))"] = x_cur[2][1]["($(edges_changed[i][1]),$(edges_changed[i][2]))"] 
        end
    end
    #w_cur = get_Wij([y_cur,w_cur])
    ##println("current sol: $(x_cur)")
    ##println("objective_value: $(obj_cur)")

    #println("neighbor_sol: ",[y_cur,w_cur,demand],"penalty: ", penalty)

    if mod == 1
        return obj_cur
    else
        return ΔObj_value
    end
    #return ([y_cur,w_cur,demand], penalty)
end

neighbor_sol (generic function with 1 method)

In [24]:
function initial_temperature(x_cur, m0 ,M, input, g)

    Δf = 0
    num = 0
    m1 = 0
    m2 = 0
    y_cur = f(x_cur, input[6])
    for i in 1:m0
        ΔObj_value = neighbor_sol(x_cur, M, input[3], g, y_cur, input[6], 0, 0) #neighbor_sol(x_cur, M, C, g, obj_cur, coeffieient, T_cur, mod)
        
        if ΔObj_value > 0 
            Δf = Δf + ΔObj_value
            num = num + 1
            m2 = m2 + 1
        else
            m1 = m1 + 1
        end
        #if f(x_new,input[6],penalty) > f(x0, input[6], 0)
            #println("m2: ", m2)
        #    Δf = Δf + (f(x_new,input[6],penalty)-f(x0,input[6],0))
        #    num = num + 1
        #    m2 = m2 + 1
        #else
            #println("m1: ",m1)
        #    m1 = m1 + 1
        #end  
    end

    Δf_avg = Δf/num
    T0 = Δf_avg/log(m2/(m2*accept_rate-m1*(1-accept_rate)))   

    return T0
     
end

initial_temperature (generic function with 1 method)

In [25]:
function Simulated_annealing(x0, T0, C, coefficient, M, g) #L = transition number
    
    y0 = f(x0,coefficient) #f(x_cur, coefficient, penalty)
    x_opt = x0
    y_opt = y0
    x_cur = x0
    y_cur = y0
    T = T0

    while(stop(T)==0)
        for i in 1:L
            y_cur = neighbor_sol(x_cur, M, C, g, y_cur, coefficient, T, 1) #neighbor_sol(x_cur, M, C, g, , coeffieient, T_cur, mod)
            #y_new = f(x_new,coefficient, penalty)obj_cur
            #Δy = y_new - y_cur
            #if Δy <= 0 || rand() < exp(-Δy/T)
            #    x_cur = x_new    
            #    y_cur = y_new
            #end
            ##println("x_cur: $(x_cur)")
            ##println("y_cur: $(y_cur)")
            if y_cur < y_opt
                x_opt = x_cur
                y_opt = y_cur
            end
        end
        T = α*T
    end

    return (x_opt, y_opt)
end

Simulated_annealing (generic function with 1 method)

In [26]:
for i in 1:size(data)[1]
    input = read_Kth_data(data,i)
    g = support_graph(input)
    sol_opt = Vector{Tuple{Any,Float64}}()

    for M = 0:ceil(sum(input[4])/input[3])-1
        #x_raw = initial_sol_min_subset(input,M, input[3])
        x_raw = initial_sol_max_subset(input,M, input[3])
        x0 = [[x_raw[1],deepcopy(x_raw[1])],[x_raw[2],deepcopy(x_raw[2])],x_raw[3]]
        T0 = initial_temperature(x0,L,M,input, g )
        push!(sol_opt, Simulated_annealing(x0,T0,input[3], input[6], M, g))
    end

    for i in 1:length(sol_opt)
        println("Obj_value: ", sol_opt[i][2])
        println("demand: ", sol_opt[i][1][3])
    end
    println("")
end


Obj_value: 2.0
demand: 4
Obj_value: 3.0
demand: 21
Obj_value: 3.0
demand: 21



In [27]:
#input = read_Kth_data(data,1)
#g = support_graph(input)
#M = 1
#x_raw = initial_sol_less_degree(input, M, input[3]) # [sol_initial_y,sol_initial_w,sum_demand]
#x0 = [[x_raw[1],deepcopy(x_raw[1])],[x_raw[2],deepcopy(x_raw[2])],x_raw[3]]
#T0 = initial_temperature(x0,L,M,input,g)
#typeof(x0)
#typeof(f(x0,input[6]))
#i = 1
#while T0 > 0.01
#    i = i+1
#    T0 = 0.95*T0
#end

#println(i)